In [3]:
import os
import json
import re
from dotenv import load_dotenv
from openai import OpenAI

# 환경 설정
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI()

In [5]:
import random

complex2 = ["구체적으로(ex. '기술적 기여도 평가에서 A/B 테스트 도입이 긍정적인 결과를 가져온 사례에 대해 자세히 설명해 주실 수 있나요?', '문제 해결 능력 강화를 위한 후속 조치 제안 중 '문제 해결 세션'의 구체적인 진행 방식은 어떻게 돼?','프론트엔드팀의 추가 인력 투입이나 교육 실행 시 예상되는 효과나 리스크는 무엇이 있을까?')", "아주 짧고 간결하게 5~10글자 내로(ex. '백엔드 팀 업무 알려줘', '임원 승계 문서 찾아줘', '프론트엔드 기술 스택 알려줘봐')"]

def pick_complex2() -> str:
    return random.choices(complex2)[0]

pick_complex2()

"구체적으로(ex. '기술적 기여도 평가에서 A/B 테스트 도입이 긍정적인 결과를 가져온 사례에 대해 자세히 설명해 주실 수 있나요?', '문제 해결 능력 강화를 위한 후속 조치 제안 중 '문제 해결 세션'의 구체적인 진행 방식은 어떻게 돼?','프론트엔드팀의 추가 인력 투입이나 교육 실행 시 예상되는 효과나 리스크는 무엇이 있을까?')"

In [7]:

def generate_questions_from_document(document_text, random):
    # 질문 생성 프롬프트 (각 질문에 번호를 붙여서 생성하도록 요청)
    prompt = (f"이 문서의 내용을 바탕으로 회사 직원이 물어볼 수 있는 구체적인 질문을 최소 1개에서 최대 3개까지 생성해 주세요. "
              f"질문의 내용은 {random} 작성해주세요."
              f"그리고 질문에는 보고서명과 보고서에서 질문하는 파트에 대한 내용이 들어가있어야 합니다."
              f"문서에 없는 내용에 대한 질문은 절대 생성하지 마세요."
              f"각 질문은 '질문 1: [질문 내용]', '질문 2: [질문 내용]' 형식으로 작성해 주세요. "
              f"예시: '고객지원팀 팀 성과 보고서에서 콘텐츠 생상량에 대한 KPI 달성 현황은 어떤가요?' "
              f"문서 내용: {document_text}")

    # OpenAI API 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1500,  # 더 긴 응답을 받을 수 있도록 확장
        temperature=0.2  # temperature 설정
    )

    # 생성된 질문 반환
    return response.choices[0].message.content.strip()

def process_directory(directory_path, output_jsonl):
    # 지정된 폴더 내 모든 파일 순회
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            # 텍스트 파일만 처리
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    document_text = f.read()
                    # 문서로부터 질문 생성
                    random = pick_complex2()
                    questions = generate_questions_from_document(document_text, random)

                    # '질문 1:', '질문 2:' 형식으로 질문을 정규식으로 추출
                    question_list = re.findall(r'질문 \d+: (.*)', questions)

                    # 생성된 질문을 JSONL 형식으로 저장
                    with open(output_jsonl, 'a', encoding='utf-8') as jsonl_file:
                        for question in question_list:
                            # 각 질문을 하나씩 JSONL 파일에 저장
                            json.dump({"question": question.strip()}, jsonl_file, ensure_ascii=False)
                            jsonl_file.write('\n')


In [12]:
# 예시 실행
directory_path = "./COMPANY_DATA/cto"  # 순회할 폴더 경로
output_jsonl = "company_questions_cto.jsonl"  # 결과를 저장할 JSONL 파일 경로
process_directory(directory_path, output_jsonl)

In [13]:
# 예시 실행
directory_path = "./COMPANY_DATA/backend"  # 순회할 폴더 경로
output_jsonl = "company_questions_backend.jsonl"  # 결과를 저장할 JSONL 파일 경로
process_directory(directory_path, output_jsonl)

In [14]:
# 예시 실행
directory_path = "./COMPANY_DATA/frontend"  # 순회할 폴더 경로
output_jsonl = "company_questions_frontend.jsonl"  # 결과를 저장할 JSONL 파일 경로
process_directory(directory_path, output_jsonl)

In [11]:
# 예시 실행
directory_path = "./COMPANY_DATA/data_ai"  # 순회할 폴더 경로
output_jsonl = "company_questions_data_ai.jsonl"  # 결과를 저장할 JSONL 파일 경로
process_directory(directory_path, output_jsonl)